### Pseudocode

Goal: Scripted process to download HPMS shapefiles from FHWA website, compile into a single feature class, and provide output as a single zipped file geodatabase. A second goal is to publish this data as a service using the ArcGIS API for Python.

### Process

1. Create list of download urls

2. Iterate on the list, download zip, and execute main zip->shapefile->feature class conversion loop

3. Merge all feature classes into a single feature class for nation-wide HPMS

4. Provide output as zipped file geodatabase. 


### Dependencies

- 'us' module. To install, run "pip install us"
- 'arcpy' module

In [1]:
# Import needed modules
import us
import arcpy
import os
import requests
import zipfile
import io
import datetime

In [8]:
# Set general variables
workspace = r"C:\Users\albe9057\Documents\ANieto_SolutionEngineering\Projects\DOT\FHWA\HPMS_Download\Work"
outputs_dir = r"C:\Users\albe9057\Documents\ANieto_SolutionEngineering\Projects\DOT\FHWA\HPMS_Download\Outputs"

### 1. Create list of download urls

In [2]:
states_list = [state.name.lower().replace(" ", "") for state in us.states.STATES]
states_list

['alabama',
 'alaska',
 'arizona',
 'arkansas',
 'california',
 'colorado',
 'connecticut',
 'delaware',
 'districtofcolumbia',
 'florida',
 'georgia',
 'hawaii',
 'idaho',
 'illinois',
 'indiana',
 'iowa',
 'kansas',
 'kentucky',
 'louisiana',
 'maine',
 'maryland',
 'massachusetts',
 'michigan',
 'minnesota',
 'mississippi',
 'missouri',
 'montana',
 'nebraska',
 'nevada',
 'newhampshire',
 'newjersey',
 'newmexico',
 'newyork',
 'northcarolina',
 'northdakota',
 'ohio',
 'oklahoma',
 'oregon',
 'pennsylvania',
 'rhodeisland',
 'southcarolina',
 'southdakota',
 'tennessee',
 'texas',
 'utah',
 'vermont',
 'virginia',
 'washington',
 'westvirginia',
 'wisconsin',
 'wyoming']

In [3]:
# Create a state data crosswalk dictionary
state_crosswalk_dict = {
    "districtofcolumbia": 'district'
}

In [4]:
# Perform state corrections for download urls
for state in state_crosswalk_dict:
    if state in states_list:
        states_list.remove(state)
        states_list.append(state_crosswalk_dict[state])

In [5]:
# Get the baseline part of the download url
baseline_download_url = r"https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles/"
year = "2015"

In [6]:
# Create a list of download URLs
download_urls = ["{0}/{1}{2}.zip".format(baseline_download_url, state, year) for state in states_list]
download_urls   

['https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//alabama2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//alaska2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//arizona2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//arkansas2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//california2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//colorado2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//connecticut2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//delaware2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//florida2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//georgia2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//hawaii2015.zip',
 'https://www.fhwa.dot.gov/policyinformation/hpms/shapefiles//idaho2015.zip',
 'https://www.fhwa.dot.gov/policyi

### 2. Iterate on states to download and convert

In [9]:
# Create the workspace file geodatabase
workspace_gdb = arcpy.CreateFileGDB_management(workspace, "hpms_workspace").getOutput(0)

In [ ]:
# Download iteration loop
for download_url in download_urls:
    state_name = download_url.split("//")[-1].split(".")[0]
    print("Downloading {0}...".format(state_name))
    response = requests.get(download_url)
    zipDocument = zipfile.ZipFile(io.BytesIO(response.content))
    shapefile_folder = os.makedirs(os.path.join(workspace, state_name))
    zipDocument.extractall(path=shapefile_folder)

In [ ]:
response = requests.get(download_urls[0])

In [ ]:
response